# Physics-Informed Neural Network

Neste notebook resolveremos a equação de Burgers em três dimensões usando uma rede neural informada por física

A ideia é ter duas saídas para esta rede

### Configuração do Ambiente

Primeiramente, para ocnseguir montar a rede neural, será preciso preparar o ambiente com as bibliotecas externas mais importantes para a aplicação.

In [ ]:
!pip install plotly --quiet

import torch
import torch.nn as nn
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

import torch.optim as optim

np.random.seed(1)

### Domínio da Função

Para a solução, serão definidos pontos no domínios, sendo x, y, t. Nesse sentido, a equação deve ser validada nesses pontos, considerando isso.

Os pontos serão distribuídos aleatoriamente em um momento inicial. Caso a solução apresente algum problema ou tendenciosidade, será considerado mudar, entre outros aspectos, a distribuição dos pontos.

In [ ]:
def gerar_pontos_contorno(pontos_no_contorno,comprimento_x,comprimento_y,tempo_final):
  pontos_por_lado = pontos_no_contorno//6

  # Lado 1 (x = qualquer, y= 0, t = qualquer)
  x_lado1 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=comprimento_x)
  y_lado1 = 0 * np.ones((pontos_por_lado,1))
  t_lado1 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=tempo_final)

  u_lado1 = 0 * np.ones((pontos_por_lado,1))
  v_lado1 = 0 * np.ones((pontos_por_lado,1))

  # Lado 2 (x = 0, y= qualquer, t = qualquer)
  x_lado2 = 0 * np.ones((pontos_por_lado,1))
  y_lado2 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=comprimento_y)
  t_lado2 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=tempo_final)

  u_lado2 = 0 * np.ones((pontos_por_lado,1))
  v_lado2 = 0 * np.ones((pontos_por_lado,1))

  # Lado 3 (x = 1, y = qualquer, t = qualquer)
  x_lado3 = 1 * np.ones((pontos_por_lado,1))
  y_lado3 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=comprimento_y)
  t_lado3 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=tempo_final)

  u_lado3 = 0 * np.ones((pontos_por_lado,1))
  v_lado3 = 0 * np.ones((pontos_por_lado,1))

  # Lado 4 (x = qualquer, y = 1, t = qualquer)
  x_lado4 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=comprimento_x)
  y_lado4 = 1 * np.ones((pontos_por_lado,1))
  t_lado4 = np.random.uniform(size=(pontos_por_lado,1),low=0,high=tempo_final)

  u_lado4 = 0 * np.ones((pontos_por_lado,1))
  v_lado4 = 0 * np.ones((pontos_por_lado,1))

  # Condicao inicial (x = qualquer, y=qualquer, t = 0)
  x_inicial = np.random.uniform(size=(2*pontos_por_lado,1),low=0,high=comprimento_x)
  y_inicial = np.random.uniform(size=(2*pontos_por_lado,1),low=0,high=comprimento_y)
  t_inicial = 0 * np.ones((2*pontos_por_lado,1))

  u_inicial = np.sin(np.pi*x_inicial/comprimento_x)*np.cos(np.pi*x_inicial/comprimento_y)
  v_inicial = np.sin(np.pi*x_inicial/comprimento_y)*np.cos(np.pi*x_inicial/comprimento_x)

  # Juntar todos os lados
  x_todos = np.vstack((x_lado1,x_lado2,x_lado3,x_lado4,x_inicial))
  y_todos = np.vstack((y_lado1,y_lado2,y_lado3,y_lado4,y_inicial))
  t_todos = np.vstack((t_lado1,t_lado2,t_lado3,t_lado4,t_inicial))
  u_todos = np.vstack((u_lado1,u_lado2,u_lado3,u_lado4,u_inicial))
  v_todos = np.vstack((v_lado1,v_lado2,v_lado3,v_lado4,v_inicial))


  # Criar arrays X e Y
  X_contorno = np.hstack((x_todos,y_todos,t_todos))
  Y_contorno = np.hstack((u_todos,v_todos))

  return X_contorno, Y_contorno
  # X contorno - reúne os pontos
  # Y contorno - reúne velocidades

### Gerar pontos de avaliação da equação

Os pontos que serão usados para treinar a rede neural. Não sabemos a solução, mas sabemos a equação a qual obedecem.

In [ ]:
def gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y,tempo_final):
  x_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=comprimento_x)
  y_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=comprimento_y)
  t_dominio = np.random.uniform(size=(pontos_no_dominio,1),low=0,high=tempo_final)

  X_equacao = np.hstack((x_dominio,y_dominio,t_dominio))

  return X_equacao

### Geração dos pontos

Agora, com base nas condições de contorno apresentadas e no domínio interno de avaliação, vamos gerar os pontos e visualizá-los.

In [ ]:
comprimento_x = 1
comprimento_y = 1
tempo_final = 1

pontos_no_contorno = 1000
pontos_no_dominio = 2000

X_contorno, Y_contorno = gerar_pontos_contorno(pontos_no_contorno,comprimento_x,comprimento_y,tempo_final)
X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,comprimento_y,tempo_final)


Hora de visualizar a vista superior

In [ ]:
# Vista superior
#fig = plt.figure()
#ax = fig.add_subplot()
scatter_contorno = px.scatter(x=X_contorno[:,0],y=X_contorno[:,1])
scatter_equacao = px.scatter(x=X_equacao[:,0],y=X_equacao[:,1], color_discrete_sequence=['red'])
fig = go.Figure(data=scatter_contorno.data+scatter_equacao.data)
fig.update_layout(xaxis_title='x',yaxis_title='t')
fig.show()

Vista em perspectiva 3d

In [ ]:
# Vista em perspectiva
scatter_3d = px.scatter_3d(x=X_contorno[:,0].flatten(),y=X_contorno[:,1].flatten(),z=Y_contorno[:,0].flatten())
fig = go.Figure(scatter_3d)
fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))
fig.show()

### Definição da Rede Neural

Agora, hora de desenvolver a rede neural para conseguir realizar a solução do problema

In [ ]:
def criar_rede_neural(numero_de_neuronios):

  # Criar uma lista de todas as camadas
  camadas = []

  # Para cada camada, adicionar as conexões e a função de ativação
  for i in range(len(numero_de_neuronios)-1):
    camadas.append(nn.Linear(numero_de_neuronios[i],numero_de_neuronios[i+1]))
    camadas.append(nn.Tanh())

  # Remover a última camada, pois é a função de ativação
  camadas.pop()
  #camadas.pop()

  # Criar rede
  return nn.Sequential(*camadas)

Agora, hora de estruturar a arquitetura da rede, com o número correto de perceptrons em cada uma das camadas, tanto de entrada e saída como nas camadas ocultas em que serão realizados os cálculos.

In [ ]:
numero_de_neuronios = [3, 20, 20, 20, 2]

rna = criar_rede_neural(numero_de_neuronios)

print(rna)

Sequential(
  (0): Linear(in_features=3, out_features=20, bias=True)
  (1): Tanh()
  (2): Linear(in_features=20, out_features=20, bias=True)
  (3): Tanh()
  (4): Linear(in_features=20, out_features=20, bias=True)
  (5): Tanh()
  (6): Linear(in_features=20, out_features=2, bias=True)
)


### Funções de Perda

Entrando na parte mais difícil do código, em que serão definidas as funções que calculam as perdas.

As perdas são de duas naturezas distinas, associadas ao contorno e associadas à equação. Ambas devem ser computadas a fim de otimizar ao máximo a rede proposta.

Primeiramente, a perda no contorno, que é bem simples, basta fazer uma comparação direta com o valor encontrado e o esperado (já definido)

In [ ]:
def calc_perda_contorno(rna,X_contorno,Y_contorno):
  Y_predito = rna(X_contorno)
  return nn.functional.mse_loss(Y_predito, Y_contorno)

A perda associada à equação considera os gradientes calculados

In [ ]:
def calc_residuo(rna,X_equacao):
  x = X_equacao[:,0].reshape(-1, 1)
  y = X_equacao[:,1].reshape(-1, 1)
  t = X_equacao[:,2].reshape(-1, 1)

  # Dois valores preditos pela rede - velocidade u(x,y,t) e velocidade v(x,y,t)
  V = rna(torch.hstack((x, y, t)))
  u = V[:,0].reshape(-1,1)
  v = V[:,1].reshape(-1,1)

  u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]

  v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]

  u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]
  v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(u),retain_graph=True, create_graph=True)[0]

  residual_u = (u_t + u * u_x + v * u_y - 0.01/np.pi * (u_xx + u_yy))
  residual_v = (v_t + u * v_x + v * v_y - 0.01/np.pi * (u_xx + u_yy))

  return torch.cat((residual_u, residual_v), dim=1)

In [ ]:
def calc_perda_equacao(rna, X_equacao):
    R = calc_residuo(rna, X_equacao)
    residuo = torch.mean(torch.square(R))

    return residuo

Agora, a perda total associada ao passo de treinamento

In [ ]:
def calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=0.2):

  perda_contorno = calc_perda_contorno(rna,X_contorno,Y_contorno)
  perda_equacao = calc_perda_equacao(rna,X_equacao)

  perda = (1-alpha)*perda_contorno + alpha*perda_equacao

  return perda, perda_contorno, perda_equacao

### Otimizador e Tensores

Para tornar o processo mais eficiente, é importante realizar os cálculos na GPU. Para tanto, vamos transferir para lá os arrays em formato de tensor.

In [ ]:
otimizador = torch.optim.Adam(rna.parameters(),lr=0.01)
agendador = torch.optim.lr_scheduler.StepLR(otimizador, step_size=1000, gamma=0.9)
alpha = 0.1

Agora, com o otimizador definido, é hora de realizar a conversão para tensor e transferir para a GPU.

Isso será fundamental para que o código rode de maneira mais rápida. Os tensores, por outro lado, são o formato que as bibliotecas de Machine Learning utilizam para realizar as operações.

In [ ]:
X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float)
X_contorno = torch.tensor(X_contorno,dtype=torch.float)
Y_contorno = torch.tensor(Y_contorno,dtype=torch.float)

device = torch.device('cuda' if torch.cuda.is_available () else 'cpu')
X_equacao = X_equacao.to(device)
X_contorno = X_contorno.to(device)
Y_contorno = Y_contorno.to(device)
rna = rna.to(device)

### Testes iniciais do Modelo

Agora, é hora de testar para verificar se o modelo está adequado, gerando respostas que sejam interessantes para o propósito.

#### Testes iniciais

Antes de realizar um treinamento na rede, é interessante realizar alguns testes e validar se tudo está funcionando dentro das conformidades.

In [ ]:
# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in range(10):

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Mostrar resultados
  print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')

Epoca: 0, Perda: 0.05280144512653351 (Contorno: 0.05861816927790642, Equacao: 0.0004509385325945914)
Epoca: 1, Perda: 0.036077044904232025 (Contorno: 0.04004925489425659, Equacao: 0.0003271609602961689)
Epoca: 2, Perda: 0.04022698476910591 (Contorno: 0.0446784682571888, Equacao: 0.0001636534434510395)
Epoca: 3, Perda: 0.03679197281599045 (Contorno: 0.04086020216345787, Equacao: 0.00017792389553505927)
Epoca: 4, Perda: 0.03251456096768379 (Contorno: 0.03609981760382652, Equacao: 0.00024725389084778726)
Epoca: 5, Perda: 0.03223972022533417 (Contorno: 0.03578757122159004, Equacao: 0.00030908695771358907)
Epoca: 6, Perda: 0.03378148749470711 (Contorno: 0.03749487176537514, Equacao: 0.0003610443673096597)
Epoca: 7, Perda: 0.033783286809921265 (Contorno: 0.03748073801398277, Equacao: 0.0005062221316620708)
Epoca: 8, Perda: 0.032425787299871445 (Contorno: 0.03594861179590225, Equacao: 0.0007203578134067357)
Epoca: 9, Perda: 0.031377993524074554 (Contorno: 0.03475891798734665, Equacao: 0.00094

### Exibição dos Resultados

Agora, como tudo parece funcionar bem, é hora de realizar algumas épocas para treinar a rede

In [ ]:
def calcular_grid(rna, comprimento_x, comprimento_y, tempo_final, nx=101, ny=101,nt=101):

    # Definir grid
    x = np.linspace(0.,comprimento_x,nx)
    y = np.linspace(0.,comprimento_y,ny)
    t = np.linspace(0.,tempo_final,nt)
    [t_grid, y_grid, x_grid] = np.meshgrid(t,y,x)
    x = torch.tensor(x_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)
    y = torch.tensor(y_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)
    t = torch.tensor(t_grid.flatten()[:,None],requires_grad=True,dtype=torch.float).to(device)

    # Avaliar modelor
    rna.eval()
    Y_pred = rna(torch.hstack((x,y,t)))

    # Formatar resultados em array
    u_pred = Y_pred.cpu().detach().numpy()[:,0].reshape(x_grid.shape)
    v_pred = Y_pred.cpu().detach().numpy()[:,1].reshape(x_grid.shape)

    return x_grid, y_grid, t_grid, u_pred, v_pred

In [ ]:
# Calcular valores da função e gerar grids
x_grid, y_grid, t_grid, u_pred, v_pred = calcular_grid(rna, comprimento_x, comprimento_y, tempo_final)

Agora, hora de treinar a rede

In [ ]:
numero_de_epocas = 20000
perda_historico = np.zeros(numero_de_epocas)
perda_contorno_historico = np.zeros(numero_de_epocas)
perda_equacao_historico = np.zeros(numero_de_epocas)
epocas = np.array(range(numero_de_epocas))

# Colocar rede em modo de treinamento
rna.train()

# FAZER ITERAÇÃO
for epoca in epocas:

  # Resortear pontos
  #X_equacao = gerar_pontos_equacao(pontos_no_dominio,comprimento_x,tempo_final)
  #X_equacao = torch.tensor(X_equacao,requires_grad=True,dtype=torch.float).to(device)

  # Inicializar gradientes
  otimizador.zero_grad()

  # Calcular perdas
  perda, perda_contorno, perda_equacao = calc_perda(rna,X_contorno,Y_contorno,X_equacao,alpha=alpha)

  # Backpropagation
  perda.backward()

  # Passo do otimizador
  otimizador.step()
  agendador.step()

  # Guardar logs
  perda_historico[epoca] = perda.item()
  perda_contorno_historico[epoca] = perda_contorno.item()
  perda_equacao_historico[epoca] = perda_equacao.item()

  if epoca%500==0:
    print(f'Epoca: {epoca}, Perda: {perda.item()} (Contorno: {perda_contorno.item()}, Equacao: {perda_equacao.item()})')



Epoca: 0, Perda: 0.03143962845206261 (Contorno: 0.034801218658685684, Equacao: 0.0011852956376969814)
Epoca: 500, Perda: 0.00884431041777134 (Contorno: 0.006533203646540642, Equacao: 0.029644273221492767)
Epoca: 1000, Perda: 0.0069690002128481865 (Contorno: 0.005127409938722849, Equacao: 0.023543309420347214)
Epoca: 1500, Perda: 0.0054238708689808846 (Contorno: 0.004137209616601467, Equacao: 0.017003819346427917)
Epoca: 2000, Perda: 0.0038165682926774025 (Contorno: 0.0026787943206727505, Equacao: 0.014056533575057983)
Epoca: 2500, Perda: 0.0029308320954442024 (Contorno: 0.002019240753725171, Equacao: 0.011135153472423553)
Epoca: 3000, Perda: 0.002190712373703718 (Contorno: 0.0014937359374016523, Equacao: 0.008463501930236816)
Epoca: 3500, Perda: 0.0017209744546562433 (Contorno: 0.0011445910204201937, Equacao: 0.006908424664288759)
Epoca: 4000, Perda: 0.0014860033988952637 (Contorno: 0.0009698924841359258, Equacao: 0.006131001748144627)
Epoca: 4500, Perda: 0.001316967885941267 (Contorno

Agora, vamos plotar os grids com a solução da equação completa

In [ ]:
# Calcular valores da função e gerar grids
x_grid, y_grid, t_grid, u_pred, v_pred = calcular_grid(rna, comprimento_x, comprimento_y, tempo_final)

# Plotar figura
fig = go.Figure(data=[go.Surface(x=x_grid, y=t_grid, z=u_pred)])

fig.update_layout(scene=dict(aspectratio=dict(x=1.5, y=1.5, z=0.5)))
fig.show()

In [ ]:
# Plotar histórico
fig = go.FigureWidget()
fig.add_trace(go.Scatter(x=epocas, y=perda_historico, name='Total', line=dict(color='black', width=4)))
fig.add_trace(go.Scatter(x=epocas, y=perda_contorno_historico, name='Contorno', line=dict(color='blue', width=2)))
fig.add_trace(go.Scatter(x=epocas, y=perda_equacao_historico, name='Equacao', line=dict(color='red', width=2)))
fig.update_yaxes(type="log")
fig.show(renderer="colab")